### Analyze Stock Trading
Using historical data with one day / one min interval

In [ ]:
import pandas as pd

In [ ]:
def print_trades(trades):
    # Print the trades in a readable format
    print("The trades were made at these times:")
    for i, trade in enumerate(trades, start=1):
        print(f'Trade {i}, Bought at {trade[0]}, sold at {trade[1]}, profit is {trade[2]}')

In [ ]:
# Set source data file
file='data/day/GOOG/data.csv'
# file='data/min/GOOG/data copy.csv'

# Read the CSV data from a file
df = pd.read_csv(file)

In [ ]:
# Use the 'close' column for the stock prices
prices = df['Close']

# Get the timestamps
timestamps = pd.to_datetime(df['Datetime'])

##### Max Profit Unlimited Trades

In [ ]:
import pandas as pd
from typing import List, Tuple

def find_increasing_sequences(prices: pd.Series, timestamps: pd.Series) -> List[Tuple[str, float, str, float, float]]:
    """
    Find all increasing sequences in stock prices.
    
    Parameters:
    prices (pd.Series): Series of stock closing prices.
    timestamps (pd.Series): Series of timestamps corresponding to the prices.
    
    Returns:
    List[Tuple[str, float, str, float, float]]: List of increasing sequences, each represented by a tuple 
    containing the start time, start value, end time, end value, and value difference, with timestamps formatted as strings.
    """
    
    # List to store the sequences
    sequences = []
    
    # Variables to track the start of the current sequence
    start_idx = None
    start_value = None
    
    for i in range(1, len(prices)):
        if prices.iloc[i] > prices.iloc[i - 1]:
            if start_idx is None:
                start_idx = i - 1
                start_value = prices.iloc[i - 1]
        else:
            if start_idx is not None:
                sequences.append((
                    timestamps[start_idx].strftime('%Y-%m-%d %H:%M:%S'),
                    float(start_value),
                    timestamps[i - 1].strftime('%Y-%m-%d %H:%M:%S'),
                    float(prices.iloc[i - 1]),
                    float(prices.iloc[i - 1] - start_value)
                ))
                start_idx = None
                start_value = None
    
    # Check if there is a sequence that ends at the last element
    if start_idx is not None:
        sequences.append((
            timestamps[start_idx].strftime('%Y-%m-%d %H:%M:%S'),
            float(start_value),
            timestamps[len(prices) - 1].strftime('%Y-%m-%d %H:%M:%S'),
            float(prices.iloc[len(prices) - 1]),
            float(prices.iloc[len(prices) - 1] - start_value)
        ))
    
    # Sort sequences by the value difference in descending order
    sorted_sequences = sorted(sequences, key=lambda row: row[4], reverse=True)
    
    return sorted_sequences


In [ ]:
# Calculate the maximum profit for day trading
sequences = find_increasing_sequences(prices, timestamps)

In [ ]:
print(f'The maximum profit from day trading is: {sequences[0][4]}')
print(f'The total number of trades is: {len(sequences)}')
for seq in sequences:
    print(seq)

##### Max Profit Predefined number of Trades

In [ ]:
max_trades = 10
trades_limited = sequences[:max_trades]
profit_limited_trades = sum(row[4] for row in trades_limited)

In [ ]:
print(f'The maximum profit with {max_trades} trades is: {profit_limited_trades}')
for trade in trades_limited:
    print(trade)

#### Comparing

In [ ]:
# Set source data file
file='data/day/GOOG/data.csv'
# file='data/min/GOOG/data copy.csv'

# Read the CSV data from a file
df = pd.read_csv(file)

In [ ]:
# Convert the 'Datetime' column to datetime
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Set 'Datetime' column as the index
df.set_index('Datetime', inplace=True)

# Group by month
grouped = df.groupby(pd.Grouper(freq='ME'))

# Group by day
# grouped = df.groupby(pd.Grouper(freq='D'))

In [ ]:
# Save each group to a separate CSV file
for name, group in grouped:
    # date_str = name.strftime('%Y-%m')
    date_str = name.strftime('%Y-%m-%d')
    # group.to_csv(f'{date_str}.csv')

    print()
    print("----------------------------------------------------------------")
    print(date_str)
    print()
    
    prices = group['Close']
    timestamps = group.index

    # Calculate the maximum profit for day trading
    sequences = find_increasing_sequences(prices, timestamps)
    total_num_of_trades = len(sequences)
    print(f'The total number of trades is: {total_num_of_trades}')
    if total_num_of_trades > 0:
        print(f'The maximum profit from day trading is: {sequences[0][4]}')
    # print_trades(trades)

    max_trades = 10
    trades_limited = sequences[:max_trades]
    profit_limited_trades = sum(row[4] for row in trades_limited)
    print(f'The maximum profit with {total_num_of_trades} trades is: {profit_limited_trades}')
    for trade in trades_limited:
        print(trade)